In [9]:
# Initial imports
import os
import pandas as pd
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
# Reading the News API key enviroment variable
api_key = os.getenv("crypto_news_key")

In [11]:
print(f"API key:{type(api_key)}")

API key:<class 'str'>


In [17]:
#Retrieve the latest news events and eventIDs. Events are important headlines that are receiving a significant amount of news coverage.
crypto_events = 'https://cryptonews-api.com/api/v1/events?&yeartodate&token='+ api_key

In [18]:
response_crypto_events = requests.get(crypto_events)

In [19]:
response_crypto_events

<Response [200]>

In [20]:
raw_crypto_events = response_crypto_events.json()


In [21]:
raw_crypto_events

{'data': [{'event_name': 'Meitu Buys $50 Million worth of Bitcoin and Ethereum',
   'event_id': 'C7',
   'news_items': 6,
   'date': 'Thu, 18 Mar 2021 20:35:00 -0400',
   'tickers': ['BTC', 'ETH']},
  {'event_name': 'Lindsay Lohan to Release Exclusive NFT Collectibles on TRON',
   'event_id': 'C6',
   'news_items': 4,
   'date': 'Thu, 18 Mar 2021 14:16:00 -0400',
   'tickers': ['TRX']},
  {'event_name': 'Bank of America slams Bitcoin in research note',
   'event_id': 'C5',
   'news_items': 5,
   'date': 'Thu, 18 Mar 2021 09:15:00 -0400',
   'tickers': ['BTC']},
  {'event_name': 'Morgan Stanley to offer Bitcoin to their richest investors',
   'event_id': 'C4',
   'news_items': 16,
   'date': 'Wed, 17 Mar 2021 13:32:00 -0400',
   'tickers': ['BTC']},
  {'event_name': "US judge denies request to add XRP holders as intervenors in SEC's lawsuit",
   'event_id': 'C3',
   'news_items': 5,
   'date': 'Wed, 17 Mar 2021 11:34:00 -0400',
   'tickers': ['XRP']},
  {'event_name': "VISA says it's de

In [22]:
events_df = pd.DataFrame(data = raw_crypto_events['data'])

In [24]:
events_df.tail()

,event_name,event_id,news_items,date,tickers
45,MicroStrategy Scoops Up $1 Billion Worth of Bi...,B62,17,"Wed, 24 Feb 2021 10:49:00 -0500",[BTC]
46,Square announces $170M bitcoin buy,B61,14,"Tue, 23 Feb 2021 19:57:00 -0500",[BTC]
47,US Secretary of Treasure Yellen Says Bitcoin i...,B60,6,"Tue, 23 Feb 2021 12:06:00 -0500",[BTC]
48,Tether And Bitfinex Settle With the New York A...,B59,14,"Tue, 23 Feb 2021 11:58:00 -0500",[USDT]
49,MoneyGram suspends partnership with Ripple,B58,11,"Mon, 22 Feb 2021 15:49:00 -0500",[XRP]


In [11]:
# Create the ETH sentiment scores DataFrame
sentiments = []

for data in raw_crypto_events['data']:
    try:
        date = data['date']
        text = data["event_name"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        tickers = data['tickers']
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "date": date,
            "text": text,
            "tickers":tickers,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
events  = pd.DataFrame(sentiments)


In [12]:
events.head()

,date,text,tickers,Compound,Positive,Negative,Neutral
0,"Thu, 18 Mar 2021 14:16:00 -0400",Lindsay Lohan to Release Exclusive NFT Collect...,[TRX],0.1280,0.158,0.000,0.842
1,"Thu, 18 Mar 2021 09:15:00 -0400",Bank of America slams Bitcoin in research note,[BTC],0.0000,0.000,0.000,1.000
2,"Wed, 17 Mar 2021 13:32:00 -0400",Morgan Stanley to offer Bitcoin to their riche...,[BTC],0.5267,0.298,0.000,0.702
3,"Wed, 17 Mar 2021 11:34:00 -0400",US judge denies request to add XRP holders as ...,[XRP],-0.5719,0.000,0.299,0.701
4,"Wed, 17 Mar 2021 11:09:00 -0400",VISA says it's developing a Bitcoin and crypto...,[BTC],0.0000,0.000,0.000,1.000


In [13]:
events.describe()

,Compound,Positive,Negative,Neutral
count,50.000000,50.000000,50.000000,50.000000
mean,0.035376,0.069980,0.032540,0.897480
std,0.244049,0.127299,0.084096,0.136491
min,-0.571900,0.000000,0.000000,0.417000
25%,0.000000,0.000000,0.000000,0.795000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.108900,0.158750,0.000000,1.000000
max,0.718400,0.583000,0.316000,1.000000


In [14]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [15]:
# Instantiate the lemmatizer
wnl = WordNetLemmatizer() 

# Create a list of stopwords
stop = stopwords.words('english')

# Expand the default stopwords list if necessary
stop.append("u")
stop.append("it'")
stop.append("'s")
stop.append("n't")
stop.append('…')
stop.append("\`")
stop.append('``')
stop.append('char')
stop.append("''")
stop = set(stop)

In [16]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    words = word_tokenize(text)

    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))
    
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))
    
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in stop, words))
    
    # Lemmatize Words into root words
    tokens = [wnl.lemmatize(word) for word in words]
    
    return tokens


In [18]:
events['tokens']=events.text.apply(tokenizer)

In [19]:
events.head()

,date,text,tickers,Compound,Positive,Negative,Neutral,tokens
0,"Thu, 18 Mar 2021 14:16:00 -0400",Lindsay Lohan to Release Exclusive NFT Collect...,[TRX],0.1280,0.158,0.000,0.842,"[Lindsay, Lohan, Release, Exclusive, NFT, Coll..."
1,"Thu, 18 Mar 2021 09:15:00 -0400",Bank of America slams Bitcoin in research note,[BTC],0.0000,0.000,0.000,1.000,"[Bank, America, slam, Bitcoin, research, note]"
2,"Wed, 17 Mar 2021 13:32:00 -0400",Morgan Stanley to offer Bitcoin to their riche...,[BTC],0.5267,0.298,0.000,0.702,"[Morgan, Stanley, offer, Bitcoin, richest, inv..."
3,"Wed, 17 Mar 2021 11:34:00 -0400",US judge denies request to add XRP holders as ...,[XRP],-0.5719,0.000,0.299,0.701,"[US, judge, denies, request, add, XRP, holder,..."
4,"Wed, 17 Mar 2021 11:09:00 -0400",VISA says it's developing a Bitcoin and crypto...,[BTC],0.0000,0.000,0.000,1.000,"[VISA, say, developing, Bitcoin, crypto, busin..."
